$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Final Project for CS445, Spring, 2019

Isaac Law

## Introduction

For this project I am going to create a program that uses Reinforcement Learning to play a simple video game. I developed a simple game in which a sheepdog tries to push the sheep into their pen. The mechanics of the game are the equivalent of particles that repel each other; the sheep flee away from the dog.  To successfully play the game the model will need to learn how to position the player behind the sheep and push them towards the pen.  The model will also need to learn how to handle map elements such as corners and walls.  Corners are a special case because the sheep can not be pushed out of the corners.

## Set up

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import random


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from game_assets import GameMap
game_map = GameMap('map_rl.txt')

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
tf.reset_default_graph()

input_state = tf.placeholder(shape=[None, 6], dtype=tf.float32)

hidden_one = slim.fully_connected(input_state, 2, activation_fn=tf.nn.tanh)
#hidden_two = slim.fully_connected(hidden_one, 2, activation_fn=tf.nn.tanh)

q_est = slim.fully_connected(hidden_one, 1, activation_fn=None)
print_q_est = tf.print(q_est)
q_estn = tf.cast(q_est, dtype=tf.float32, name="Scalar q_est")
print_q_estn = tf.print(q_estn)

def action_picker(sess, state, epsilon):
    '''
    1 represents standing still.
    2-5 are N, E, S, W
    where N = [0, -1]
    '''    
    actions = [i for i in range(1, 6)]
    
    if True:# np.random.uniform() < epsilon:
        action = np.random.choice(actions)
    else:
        random_actions = random.sample(actions, len(actions))
        Qs = [NNETQ.use(np.array([[state, a]])) for a in random_actions]
        ai = np.argmax(Qs)
        action = random_actions[ai]
    Q = sess.run([q_estn], feed_dict={input_state: state.reshape(1, -1)})
    Q = Q[0]
    return action, Q

def reinforcement(state):
    '''
    The score is:
       # sheep in pen * 10
    - dis(sheep, pen)
    '''
    global game_map
    saved_state, _ = game_map.get_state()
    game_map.set_state(state)
    score = game_map.get_score()
    game_map.set_state(saved_state)
    return score

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def sample_game(sess, cur_state, num_samples, epsilon):
    global game_map
    num_sheep = len(game_map.sheep)
    num_cols = 2 * (1 + num_sheep) + 1
    X = np.zeros((num_samples, num_cols))
    R = np.zeros((num_samples, 1))
    Qn = np.zeros((num_samples, 1))
    
    action, _ = action_picker(sess, cur_state, epsilon)
    for step in range(num_samples):
        game_map.advance_moves(action, 20)
        next_state, is_fin = game_map.get_state()
        ref_next = reinforcement(next_state)
        action_next, q_next = action_picker(sess, next_state, epsilon)
        
        cur_state = cur_state.reshape(1, -1)
        cur_state = np.append(cur_state, action)
        X[step, :] = cur_state
        R[step, 0] = ref_next
        Qn[step, 0] = q_next
        
        cur_state, action = next_state, action_next
    return X, R, Qn

In [8]:
def run_training():
    #optimizer = tf.train.GradientDescentOptimizer(1e-3)
    #train_op = optimizer.minimize(loss)

    init = tf.global_variables_initializer()
    number_updates = 1000
    with tf.Session() as sess:
        sess.run(init)
        state, is_fin = game_map.get_state()
        samples = 50
        epsilon = .1
        X, R, Qn = sample_game(sess, state, samples, epsilon)
        print('-'*20)
        print(X)
        print(R)
        print(Qn)
run_training()

--------------------
[[140.          30.          43.82392462 158.37848863  41.68971423
  243.2788932    1.        ]
 [140.          30.          43.82392462 158.37848863  34.61864642
  236.20782539   1.        ]
 [140.          30.          43.82392462 166.87848863  30.70984084
  229.17445372   1.        ]
 [140.          30.          43.82392462 176.87848863  30.13056459
  222.64920508   4.        ]
 [140.          70.          43.82392462 186.87848863  30.59575859
  216.79917082   1.        ]
 [140.          70.          43.82392462 187.87848863  30.17567241
  211.69285784   1.        ]
 [140.          70.          43.82392462 187.87848863  30.21163437
  207.32881311   1.        ]
 [140.          70.          43.82392462 187.87848863  30.0170922
  203.67872407   1.        ]
 [140.          70.          43.82392462 187.87848863  30.0341844
  203.51250188   5.        ]
 [100.          70.          42.82392462 187.87848863  30.0341844
  203.51250188   2.        ]
 [100.          30.   

## Methods

The reason I want to do this project is because it reminds me of Google's AlphaGo project.  AlphaGo was one of the first machine learning programs I learned about, and it fascinated me.  The program was able to beat the world's best Go player, not by being taught the best strategies, but by playing itself over and over again and improving each time.  I am really interested in the ability of machine learning to discover knowledge and strategies on its own. It seems that an algorithm teaching itself checkers is discovering knowledge, while algorithms trying to predict data sets are simply matching patterns.

I'm currently planning on referencing this article:
https://towardsdatascience.com/how-to-teach-an-ai-to-play-games-deep-reinforcement-learning-28f9b920440a

I will convert the machine learning code used to play snake to code to play checkers with.  Then I will develop a process to duplicate the neural network to have two different players.  These players will face off in the checkers game, and the winner will move on to the next round.  In the next round I will mutate new opponents from the previous winner.  With enough iterations this "survival of the fittest" should produce a competent model of a checkers player.


## Possible Results

At this point I think there are two possible results.

The first is that I won't have enough computing power or time to train a competent checkers player.  This is not unlikely as there are a number of unknowns I will need to overcome. 

The second possibility is that I train a decent checkers player.  If I can get the evolutionary training process working, then there will probably be a number of basic moves that the machine learning model will be able to pick up. It seems unlikely though that the project will succeed at such a level that it could be classified as an expert checkers player.  

## Timeline
- April 19th, 2019.   Mechanics of checkers game will be set up
- April 26th, 2019.   Game heuristics and loss and update functions are created
- May 1st, 2019.   Genetic based training is working, time to tweak as required
- May 8th, 2019.   Final results will be analyzed and report completed